# Лабораторная работа на тему "Прямые методы решения слау"

### Вариант 10
Эскалаторный метод решения слау

##### описание метода
Это специальный быстрый алгоритм для решения систем с трехдиагональной матрицей вида:
$$
\begin{bmatrix}
b_1 & c_1 & 0 & \cdots & 0 \\
a_2 & b_2 & c_2 & \cdots & 0 \\
0 & a_3 & b_3 & \cdots & 0 \\
\cdots & \cdots & \cdots & \cdots & \cdots \\
0 & 0 & 0 & a_n & b_n
\end{bmatrix}
×
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\cdots \\
x_n
\end{bmatrix}
=
\begin{bmatrix}
d_1 \\
d_2 \\
d_3 \\
\cdots \\
d_n
\end{bmatrix}
$$

In [201]:
from typing import Literal
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import sympy as sp

In [202]:
x_i_sym = sp.Symbol('x')  # x_i

In [203]:
def walking(
    matrix: np.ndarray,
    d_array: np.ndarray,
    k: int,
    direction: Literal["top", "bottom"]
) -> list[tuple[int, int]]:
    n = matrix.shape[0]
    t_x_i_save = [None for _ in range(n)]
    """
    первый этап:

    сверху:
    выражаем в первом уравнении x_1 через x_2
    b_1 x_1 + c_1 x_2 = d_1
    x_1 = d_1 / b_1 - c_1 x_2 / b_1
    x_i = d_1 / b_1 - c_1 x_ip1 / b_1
    для следующего - x_im1 = d_1 / b_1 - c_1 x_i / b_1

    снизу:
    выражаем в первом уравнении x_n через x_nm1
    a_n x_nm1 + b_n x_n = d_n
    x_n = d_n / a_n - b_n x_nm1 / a_n
    x_i = d_n / a_n - b_n x_im1 / a_n
    для следующего - x_ip1 = d_n / a_n - b_n x_i / a_n
    """
    match direction:
        case "top":
            b_1, c_1, d_1 = matrix[0, 0], matrix[0, 1], d_array[0]
            t_x_i = (d_1/b_1, -c_1/b_1)  # (y, z)
            t_x_i_save[1] = t_x_i
        case "bottom":
            a_n, b_n, d_n = matrix[n-1,n-2], matrix[n-1, n-1], d_array[n-1]
            t_x_i = (d_n/b_n, -a_n/b_n)  # (y, z)
            t_x_i_save[n-2] = t_x_i
    

    """
    второй этап:
    сверху:
    итерируемся и на каждой итерации выражаем x_i через x_ip1
    a_i x_im1 + b_i x_i + c_i x_ip1 = d_i
    a_i t(x_i) + b_i x_i + c_i x_ip1 = d_i
    a_i t(x_i) + b_i x_i = y + z x_i  (z - коэф)
    y + z x_i + c_i x_ip1 = d_i
    x_i = (d_i - y) / z - c_i x_ip1 / z
    для следующего - x_im1 = (d_i - y) / z - c_i x_i / z

    снизу:
    итерируемся и на каждой итерации выражаем x_i через x_im1
    a_i x_im1 + b_i x_i + c_i x_ip1 = d_i
    a_i x_im1 + b_i x_i + c_i t(x_i) = d_i
    b_i x_i + c_i t(x_i) = y + z x_i  (z - коэф)
    a_i x_im1 + y + z x_i = d_i
    x_i = (d_i - y) / z - a_i x_im1 / z
    для следующего - x_ip1 = (d_i - y) / z - a_i x_i / z
    """

    match direction:
        case "top":
            for i in range(1, k):
                a_i, b_i, c_i, d_i = matrix[i,i-1], matrix[i,i], matrix[i,i+1], d_array[i]
                y, z = t_x_i
                y, z = (a_i * y, a_i * z + b_i)
                t_x_i = ((d_i-y)/z, -c_i/z)
                t_x_i_save[i+1] = t_x_i
        case "bottom":
            for i in range(n-2, k, -1):
                a_i, b_i, c_i, d_i = matrix[i,i-1], matrix[i,i], matrix[i,i+1], d_array[i]
                y, z = t_x_i
                y, z = (c_i * y, c_i * z + b_i)
                t_x_i = ((d_i-y)/z, -a_i/z)
                t_x_i_save[i-1] = t_x_i
    return t_x_i_save

    

In [204]:
def eval_k(
    matrix: np.ndarray,
    d_array: np.ndarray,
    k: int,
    top_saving_t_x_i: list[tuple[int, int]],
    bottom_saving_t_x_i: list[tuple[int, int]],
):
    a_k, b_k, c_k, d_k = matrix[k, k-1], matrix[k, k], matrix[k, k+1], d_array[k]

    top_y, top_z = top_saving_t_x_i[k]
    bottom_y, bottom_z = bottom_saving_t_x_i[k]

    top_y, top_z = a_k * top_y, a_k * top_z
    bottom_y, bottom_z = c_k * bottom_y, c_k * bottom_z

    x_k = (d_k-top_y-bottom_y) / (top_z + bottom_z + b_k)
    return x_k

In [205]:
def eval(
    matrix: np.ndarray,
    d_array: np.ndarray,
    k: int,
    x_k: int,
    saving_t_x_i: list[tuple[int, int]],
    direction: Literal["top", "bottom"]
) -> list[int]:
    n = matrix.shape[0]
    result = [None for _ in range(n)]
    result[k] = x_k

    """
    третий этап:
    вверх:
    итерируемся и на каждой итерации зная какой x_ip1 вычисляем x_i
    a_i x_im1 + b_i x_i + c_i x_ip1 = d_i
    a_i t(x_i) + b_i x_i + c_i x_ip1_known = d_i
    a_i y + a_i z x_i + b_i x_i + c_i x_ip1_known = d_i
    x_i = (d_i - c_i x_ip1_known - a_i y) / (a_i z x_i + b_i)

    вниз:
    итерируемся и на каждой итерации зная какой x_im1 вычисляем x_i
    a_i x_im1 + b_i x_i + c_i x_ip1 = d_i
    a_i x_im1_known + b_i x_i + c_i t(x_i) = d_i
    a_i x_im1_known + b_i x_i + c_i z x_i + c_i y = d_i
    x_i = (d_i - a_i x_im1_known - c_i y) / (c_i z x_i + b_i)
    """

    match direction:
        case "top":
            for i in range(k-1, 0, -1):
                a_i, b_i, c_i, d_i = matrix[i,i-1], matrix[i,i], matrix[i,i+1], d_array[i]
                y, z = saving_t_x_i[i]
                y, z = (a_i * y, a_i * z + b_i)
                x_ip1_known = result[i+1]
                result[i] = (d_i - c_i * x_ip1_known - y) / z
        case "bottom":
            for i in range(k+1, n-1):
                a_i, b_i, c_i, d_i = matrix[i,i-1], matrix[i,i], matrix[i,i+1], d_array[i]
                y, z = saving_t_x_i[i]
                y, z = (c_i * y, c_i * z + b_i)
                x_im1_known = result[i-1]
                result[i] = (d_i - x_im1_known - y) / z

    """
    четвёртый этап:
    вверх:
    считаем x_1
    b_1 * x_1 + c_1 * x_2_known = d_1
    x_1 = (d_1 - c_1 * x_2_known) / b_1

    вниз:
    считаем x_n
    a_n * x_nm1_known + b_n * x_n = d_n
    x_n = (d_1 - a_1 * x_nm1_known) / b_1
    """

    match direction:
        case "top":
            b_1, c_1, d_1 = matrix[0,0], matrix[0,1], d_array[0]
            x_2_known = result[1]
            x_1 = (d_1 - c_1 * x_2_known) / b_1
            result[0] = x_1
        case "bottom":
            a_n, b_n, d_n = matrix[n-1,n-2], matrix[n-1,n-1], d_array[n-1]
            x_nm1_known = result[n-2]
            x_n = (d_n - a_n * x_nm1_known) / b_n
            result[n-1] = x_n
    
    return result

In [206]:
def solving(
    matrix: np.ndarray,
    d_array: np.ndarray,
    with_logging: bool = False
):
    n = matrix.shape[0]
    k = n // 2

    top_saving_t_x_i = walking(matrix, d_array, k, direction="top")
    bottom_saving_t_x_i = walking(matrix, d_array, k, direction="bottom")

    x_k = eval_k(matrix, d_array, k, top_saving_t_x_i, bottom_saving_t_x_i)

    top_result = eval(matrix, d_array, k, x_k, top_saving_t_x_i, direction="top")
    bottom_result = eval(matrix, d_array, k, x_k, bottom_saving_t_x_i, direction="bottom")

    return [i or j for i, j in zip(top_result, bottom_result)]

In [207]:
matrix = np.array([
    [1, 2, 0, 0],
    [3, 1, 2, 0],
    [0, 4, 1, 3],
    [0, 0, 5, 2],
])
d_array = np.array([
    1,
    1,
    3,
    3,
])

In [208]:
np_solving = np.linalg.solve(matrix, d_array)

In [209]:
my_solving = solving(matrix, d_array)

In [210]:
max_error = 0
for np_x, my_x in zip(np_solving, my_solving):
    max_error = max(max_error, abs(np_x - my_x))
max_error

np.float64(2.498001805406602e-16)